<a href="https://colab.research.google.com/github/subikkshas/DA6401/blob/main/DLass1Q10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define a function to build and train the model
def train_mnist_model(learning_rate, batch_size, hidden_units, activation, optimizer):

    model = Sequential([
        Flatten(input_shape=(28, 28)),  # Flatten the 28x28 input images
        Dense(hidden_units[0], activation=activation),  # First hidden layer
        Dense(hidden_units[1], activation=activation),  # Second hidden layer
        Dense(10, activation='softmax')  # Output layer
    ])


    model.compile(optimizer=optimizer(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.1, verbose=1)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_accuracy:.4f}")

    return test_accuracy

# Configuration 1: Adam optimizer, learning rate = 0.001, batch size = 128, 2-layer network (128, 64), ReLU
print("Configuration 1:")
accuracy_1 = train_mnist_model(learning_rate=0.001, batch_size=128, hidden_units=[128, 64], activation='relu', optimizer=Adam)

# Configuration 2: Adam optimizer, learning rate = 0.001, batch size = 64, 2-layer network (128, 64), ReLU
print("Configuration 2:")
accuracy_2 = train_mnist_model(learning_rate=0.001, batch_size=64, hidden_units=[128, 64], activation='relu', optimizer=Adam)

# Configuration 3: Adam optimizer, learning rate = 0.0005, batch size = 128, 2-layer network (256, 128), ReLU
print("Configuration 3:")
accuracy_3 = train_mnist_model(learning_rate=0.0005, batch_size=128, hidden_units=[256, 128], activation='relu', optimizer=Adam)

# Print the results
print(f"Configuration 1: Adam optimizer, learning rate = 0.001, batch size = 128, 2-layer network (128, 64), ReLU \nTest Accuracy: {accuracy_1:.4f}")
print(f"Configuration 2: Adam optimizer, learning rate = 0.001, batch size = 64, 2-layer network (128, 64), ReLU \nTest Accuracy: {accuracy_2:.4f}")
print(f"Configuration 3: Adam optimizer, learning rate = 0.0005, batch size = 128, 2-layer network (256, 128), ReLU \nTest Accuracy: {accuracy_3:.4f}")

Configuration 1:
Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8263 - loss: 0.6186 - val_accuracy: 0.9617 - val_loss: 0.1426
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9537 - loss: 0.1562 - val_accuracy: 0.9700 - val_loss: 0.1081
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9687 - loss: 0.1057 - val_accuracy: 0.9708 - val_loss: 0.1027
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9772 - loss: 0.0769 - val_accuracy: 0.9723 - val_loss: 0.0935
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9831 - loss: 0.0597 - val_accuracy: 0.9760 - val_loss: 0.0851
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9848 - loss: 0.0498 - val_accuracy: 0.9772 - val_loss: 0.0801
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9888 - loss: 0.0387 - val_accuracy: 0.9772 - val_loss: 0.0832
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9895 - loss: 0.0327 -